In [2]:
!pip install vectorbt
!pip install condacolab
import condacolab
condacolab.install()
!conda install -c conda-forge ta-lib
!pip install vectorbt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.41.1
    Uninstalling llvmlite-0.41.1:
      Successfully uninstalled llvmlite-0.41.1
  Attempting uninstall: numba
    Found existing installation: numba 0.58.1
    Uninstalling numba-0.58.1:
      Succes

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:11
🔁 Restarting kernel...
Channels:
 - conda-forge
Platform: linux-64
Solving environment: / - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - ta-lib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2024.2.2   |       hbcca054_0         152 KB  conda-forge
    certifi-2024.2.2           |     pyhd8ed1ab_0         157 KB  conda-forge
    libblas-3.9.0              |22_linux64_openblas          14 KB  conda-forge
    libcblas-3.

In [1]:
import vectorbt as vbt
import talib
import requests
import json
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
import time
import datetime
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, BatchNormalization
import matplotlib.pyplot as plt


In [2]:
class GetBinanceData:
    @staticmethod
    def get_data(symbol, interval, startDate=datetime.datetime(2020, 1, 1), endDate=datetime.datetime.now()):
        dataList = []
        current_date = datetime.datetime.now()
        start_date = startDate

        delta_units = GetBinanceData.convert_to_minutes(interval)
        print(delta_units)
        delta = datetime.timedelta(minutes=delta_units)

        while start_date < current_date:
            end_date = start_date + delta
            if end_date > current_date:
                end_date = current_date
            # Convert datetime objects to milliseconds
            start_ms = int(start_date.timestamp() * 1000)
            end_ms = int(end_date.timestamp() * 1000)

            start_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
            end_str = end_date.strftime('%Y-%m-%d %H:%M:%S')

            print(f"Fetching data from {start_str} to {end_str}")
            data = GetBinanceData.get_historical_data(symbol, interval, start_ms, end_ms)
            if data:
                dataList.extend(data)
            start_date = end_date
            time.sleep(1)

        return dataList

    @staticmethod
    def get_historical_data(symbol, interval, start_time, end_time):
        url = f"https://api.binance.us/api/v3/klines"
        params = {
            'symbol': symbol.upper(),
            'interval': interval,
            'startTime': start_time,
            'endTime': end_time,
            'limit': 1000
        }

        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            return data
        else:
            print(f"Failed to fetch data. Status code: {response.status_code}")
            return None

    @staticmethod
    def convert_to_minutes(time_str):
        time_mapping = {
            'm': 1,
            'h': 60,
            'd': 1440,
            'w': 10080,
            'M': 43200
        }

        num = int(time_str[:-1])
        unit = time_str[-1]

        return (num * time_mapping[unit])*1000

    def data_to_dataframe(data):
      # Define the column names based on Binance API response
      columns = [
          "Open Time", "Open", "High", "Low", "Close", "Volume",
          "Close Time", "Quote Asset Volume", "Number of Trades",
          "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore"
      ]
      # Convert the data to a pandas DataFrame
      df = pd.DataFrame(data, columns=columns)
      # Convert timestamp columns to datetime
      df["Open Time"] = pd.to_datetime(df["Open Time"], unit='ms')
      df["Close Time"] = pd.to_datetime(df["Close Time"], unit='ms')
      # Convert numeric columns to float
      numeric_columns = ["Open", "High", "Low", "Close", "Volume", "Quote Asset Volume",
                        "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume"]
      df[numeric_columns] = df[numeric_columns].astype(float)
      return df

    @staticmethod
    def get_data_dataFrame(symbol, interval, startDate=datetime.datetime(2020, 1, 1), endDate=datetime.datetime.now()):
        data = GetBinanceData.get_data(symbol, interval, startDate, endDate)
        return GetBinanceData.data_to_dataframe(data)

In [10]:
class IndicatorFactory:
    @staticmethod
    def calculate_sma(df, period, smakey=''):
        df['SMA' + smakey] = talib.SMA(df['Close'], timeperiod=period)
        return df

    @staticmethod
    def calculate_rsi(df, period):
        df['RSI'] = talib.RSI(df['Close'], timeperiod=period)
        return df

    @staticmethod
    def calculate_hma(df, period):
        df['WMA_half'] = talib.WMA(df['Close'], timeperiod=int(period / 2))
        df['WMA_full'] = talib.WMA(df['Close'], timeperiod=period)
        df['HMA'] = talib.WMA(2 * df['WMA_half'] - df['WMA_full'], timeperiod=int(np.sqrt(period)))
        return df

    @staticmethod
    def calculate_sar(df, acceleration=0.02, maximum=0.2):
        df['SAR'] = talib.SAR(df['High'], df['Low'], acceleration=acceleration, maximum=maximum)
        return df

    @staticmethod
    def calculate_adx(df, period=14):
        df['ADX'] = talib.ADX(df['High'], df['Low'], df['Close'], timeperiod=period)
        return df

    @staticmethod
    def calculate_obv(df):
        df['OBV'] = talib.OBV(df['Close'], df['Volume'])
        df['OBV_diff'] = df['OBV'].diff()
        return df

    @staticmethod
    def obv_condition(data):
        return data['OBV_diff'] > 0

    @staticmethod
    def rsi_condition(data):
        return data['RSI'] > 30

    @staticmethod
    def adx_condition(data):
        return data['ADX'].diff() > 0

    @staticmethod
    def calculate_bbands(df, period, nbdevup=2, nbdevdn=2, matype=0):
        upperband, middleband, lowerband = talib.BBANDS(df['Close'], timeperiod=period, nbdevup=nbdevup, nbdevdn=nbdevdn, matype=matype)
        df['BB_upper'] = upperband
        df['BB_middle'] = middleband
        df['BB_lower'] = lowerband
        return df

    @staticmethod
    def calculate_pivot(df):
        df['Pivot'] = (df['High'].shift(1) + df['Low'].shift(1) + df['Close'].shift(1)) / 3
        df['R1'] = 2 * df['Pivot'] - df['Low'].shift(1)
        df['S1'] = 2 * df['Pivot'] - df['High'].shift(1)
        df['R2'] = df['Pivot'] + (df['High'].shift(1) - df['Low'].shift(1))
        df['S2'] = df['Pivot'] - (df['High'].shift(1) - df['Low'].shift(1))
        df['R3'] = df['High'].shift(1) + 2 * (df['Pivot'] - df['Low'].shift(1))
        df['S3'] = df['Low'].shift(1) - 2 * (df['High'].shift(1) - df['Pivot'])
        return df
    @staticmethod
    def calculate_DarkCloudCover(df):
      df['DarkCloudCover'] = talib.CDLDARKCLOUDCOVER(df['Open'], df['High'], df['Low'], df['Close'])
      return df

    @staticmethod
    def calculate_BearishEngulfing(df):
      df['BearishEngulfing'] = talib.CDLENGULFING(df['Open'], df['High'], df['Low'], df['Close'])
      return df



In [4]:
timeperiod = '1h'
data = GetBinanceData.get_data('ethusdt', timeperiod, startDate=datetime.datetime(2022, 1, 20), endDate=datetime.datetime.now())

60000
Fetching data from 2022-01-20 00:00:00 to 2022-03-02 16:00:00
Fetching data from 2022-03-02 16:00:00 to 2022-04-13 08:00:00
Fetching data from 2022-04-13 08:00:00 to 2022-05-25 00:00:00
Fetching data from 2022-05-25 00:00:00 to 2022-07-05 16:00:00
Fetching data from 2022-07-05 16:00:00 to 2022-08-16 08:00:00
Fetching data from 2022-08-16 08:00:00 to 2022-09-27 00:00:00
Fetching data from 2022-09-27 00:00:00 to 2022-11-07 16:00:00
Fetching data from 2022-11-07 16:00:00 to 2022-12-19 08:00:00
Fetching data from 2022-12-19 08:00:00 to 2023-01-30 00:00:00
Fetching data from 2023-01-30 00:00:00 to 2023-03-12 16:00:00
Fetching data from 2023-03-12 16:00:00 to 2023-04-23 08:00:00
Fetching data from 2023-04-23 08:00:00 to 2023-06-04 00:00:00
Fetching data from 2023-06-04 00:00:00 to 2023-07-15 16:00:00
Fetching data from 2023-07-15 16:00:00 to 2023-08-26 08:00:00
Fetching data from 2023-08-26 08:00:00 to 2023-10-07 00:00:00
Fetching data from 2023-10-07 00:00:00 to 2023-11-17 16:00:00
Fe

In [5]:
df = GetBinanceData.data_to_dataframe(data)
df.head()
len(df)

20724

In [11]:
# Calculate all necessary indicators
df = IndicatorFactory.calculate_sma(df, 20, '20')
df = IndicatorFactory.calculate_sma(df, 50, '50')
df = IndicatorFactory.calculate_sma(df, 200, '200')
df = IndicatorFactory.calculate_hma(df, 20)
df = IndicatorFactory.calculate_sar(df)
df = IndicatorFactory.calculate_adx(df)
df = IndicatorFactory.calculate_obv(df)
df = IndicatorFactory.calculate_rsi(df, 14)
df = IndicatorFactory.calculate_bbands(df , 20)
df = IndicatorFactory.calculate_pivot(df)
df = IndicatorFactory.calculate_DarkCloudCover(df)
df = IndicatorFactory.calculate_BearishEngulfing(df)

In [12]:
df = df.dropna()
type(df)
df.head()
len(df)

20326

In [20]:

long_signal = (
    (df['Close'] > df['R1']) &
    (df['Close'].shift(1) <= df['R1']) &
    (df['SMA20'] > df['SMA50']) &
    (df['SMA50'] > df['SMA200']) &
    (df['SAR'] < df['Close']) &
    (df['HMA'].shift(1) < df['HMA'])

)
close_signal = (
    # (df['SMA20'] < df['SMA50']) |
    # (df['SMA50'] < df['SMA200']) |
    # (df['SAR'] > df['Close']) |
    (df['HMA'].shift(1) > df['HMA'])
    #  (df['BearishEngulfing'] < 0) |
    #  (df['DarkCloudCover'] < 0)

    )


entries = long_signal.astype(int).values
exits = close_signal.astype(int).values

portfolio = vbt.Portfolio.from_signals(df['Close'], entries=entries, exits=exits,  sl_stop=0.04,   # Stop-loss at 4%
    tp_stop=0.07,   # Take-profit at 7%
     freq=timeperiod)

# Get portfolio statistics
stats = portfolio.stats()
print(stats)

Start                                               398
End                                               20723
Period                                846 days 22:00:00
Start Value                                       100.0
End Value                                    143.834571
Total Return [%]                              43.834571
Benchmark Return [%]                          26.042481
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               20.57366
Max Drawdown Duration                 163 days 07:00:00
Total Trades                                        267
Total Closed Trades                                 267
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  35.580524
Best Trade [%]                                10.077882
Worst Trade [%]                               -3

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score

In [8]:
# Prepare the dataset for training
long_signal = (
     (df['SMA20'] > df['SMA50']) &
     (df['SMA50'] > df['SMA200']) &
     (df['SAR'] < df['Close']) &
    (df['HMA'].shift(1) < df['HMA']) &
    IndicatorFactory.obv_condition(df) &
    IndicatorFactory.rsi_condition(df) &

    IndicatorFactory.adx_condition(df)
)
close_signal = (
    (df['SMA20'] < df['SMA50']) |
    (df['SMA50'] < df['SMA200']) |
    (df['SAR'] > df['Close']) |
    (df['HMA'].shift(1) > df['HMA']) |
    ~IndicatorFactory.obv_condition(df) |
    (df['RSI'] < 70) |
    ~IndicatorFactory.adx_condition(df)
)
df['Signal'] = 0
df.loc[long_signal, 'Signal'] = 1
df.loc[close_signal, 'Signal'] = -1
df['Future_Signal'] = df['Signal'].shift(-1)  # Predict the next day's signal
df.dropna(inplace=True)

X = df[['SMA20', 'SMA50', 'SMA200', 'SAR', 'HMA']]
y = df['Future_Signal']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the decision tree model
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Generate signals with the trained model
df['Predicted_Signal'] = model.predict(df[['SMA20', 'SMA50', 'SMA200', 'SAR', 'HMA']])
df['Entry_Signal'] = df['Predicted_Signal'] == 1
df['Exit_Signal'] = df['Predicted_Signal'] == -1

# Backtest with vectorbt
portfolio = vbt.Portfolio.from_signals(
    df['Close'],
    entries=df['Entry_Signal'],
    exits=df['Exit_Signal'],
    sl_stop=0.003,
    tp_stop=0.020,
    freq=timeperiod
)

portfolio_stats = portfolio.stats()
print(portfolio_stats)
import

SyntaxError: invalid syntax (<ipython-input-8-df79d435e893>, line 59)

In [1]:
long_signal = (
     (df['SMA20'] > df['SMA50']) &
     (df['SMA50'] > df['SMA200']) &
     (df['SAR'] < df['Close']) &
    (df['HMA'].shift(1) < df['HMA'])

) and
for i in range(1, len(df)):
    if df['Close'][i] > df['Peaks'][i-1]:  # Price breaks above previous peak
        df['Buy_Signal'][i] = df['Close'][i]

# Generate sell signals
for i in range(1, len(df)):
    if df['Close'][i] < df['Troughs'][i-1]:  # Price breaks below previous trough
        df['Sell_Signal'][i] = df['Close'][i]

# long_signal = (

#      (df['SAR'] < df['Close']) &

#      (df['RSI'] < 30)

#     # IndicatorFactory.adx_condition(df)
# )
# print(long_signal)

SyntaxError: invalid syntax (<ipython-input-1-f2ebb79cf6c5>, line 7)

In [20]:
close_signal = (
    (df['SMA20'] < df['SMA50']) |
    (df['SMA50'] < df['SMA200']) |
    (df['SAR'] > df['Close']) |
    (df['HMA'].shift(1) > df['HMA']) |
    (df['BB_upper'] <= df['Close'])


    )
print(close_signal)


398      True
399      True
400      True
401      True
402      True
         ... 
20715    True
20716    True
20717    True
20718    True
20719    True
Length: 20322, dtype: bool


In [ ]:
close_signal = (

    (df['HMA'].shift(1) > df['HMA']) &
    ~IndicatorFactory.obv_condition(df) &

    ~IndicatorFactory.adx_condition(df)
)

In [ ]:
close_signal = (

    (df['SAR'] > df['Close'])  &
    (df['RSI'] < 70) &
    ~IndicatorFactory.obv_condition(df) &

    ~IndicatorFactory.adx_condition(df)
)

In [ ]:
close_signal = (

    (df['SAR'] > df['Close']) &

    ~IndicatorFactory.obv_condition(df) &

    ~IndicatorFactory.adx_condition(df)
)

In [ ]:
close_signal = (

    (df['HMA'].shift(1) > df['HMA']) &
    ~IndicatorFactory.obv_condition(df) &
    (df['RSI'] < 70) &
    ~IndicatorFactory.adx_condition(df)
)
#0.96 30m
#                                     BTC USDT 15m

# Start                                               199
# End                                              152446
# Period                               1585 days 22:00:00
# Start Value                                       100.0
# End Value                                    216.811003
# Total Return [%]                             116.811003
# Benchmark Return [%]                         704.958516
# Max Gross Exposure [%]                            100.0
# Total Fees Paid                                     0.0
# Max Drawdown [%]                              49.311411
# Max Drawdown Duration                1029 days 02:15:00
# Total Trades                                       1730
# Total Closed Trades                                1730
# Total Open Trades                                     0
# Open Trade PnL                                      0.0
# Win Rate [%]                                  33.872832
# Best Trade [%]                                10.238297
# Worst Trade [%]                               -7.014937
# Avg Winning Trade [%]                          1.233047
# Avg Losing Trade [%]                          -0.550778
# Avg Winning Trade Duration    0 days 04:26:48.020477815
# Avg Losing Trade Duration     0 days 01:55:47.465034965
# Profit Factor                                  1.128399
# Expectancy                                     0.067521
# Sharpe Ratio                                   0.838418
# Calmar Ratio                                   0.395343
# Omega Ratio                                    1.040796
# Sortino Ratio                                  1.226962
# dtype: object

"""Start                                               199
End                                               82412
Period                                856 days 09:30:00
Start Value                                       100.0
End Value                                    170.953376
Total Return [%]                              70.953376
Benchmark Return [%]                          92.375251
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              21.627252
Max Drawdown Duration                 317 days 21:30:00
Total Trades                                        885
Total Closed Trades                                 885
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                   32.99435
Best Trade [%]                                10.066941
Worst Trade [%]                               -5.460466
Avg Winning Trade [%]                          1.140201
Avg Losing Trade [%]                          -0.459675
Avg Winning Trade Duration    0 days 04:38:43.972602739
Avg Losing Trade Duration     0 days 01:58:07.689713322
Profit Factor                                  1.219007
Expectancy                                     0.080173
Sharpe Ratio                                   1.194724
Calmar Ratio                                   1.187222
Omega Ratio                                    1.059252
Sortino Ratio                                  1.798879
dtype: object
Total Return:  0.709533757724274"""

'Start                                               199\nEnd                                               82412\nPeriod                                856 days 09:30:00\nStart Value                                       100.0\nEnd Value                                    170.953376\nTotal Return [%]                              70.953376\nBenchmark Return [%]                          92.375251\nMax Gross Exposure [%]                            100.0\nTotal Fees Paid                                     0.0\nMax Drawdown [%]                              21.627252\nMax Drawdown Duration                 317 days 21:30:00\nTotal Trades                                        885\nTotal Closed Trades                                 885\nTotal Open Trades                                     0\nOpen Trade PnL                                      0.0\nWin Rate [%]                                   32.99435\nBest Trade [%]                                10.066941\nWorst Trade [%]               

In [ ]:
close_signal = (


    ~IndicatorFactory.obv_condition(df) &
    (df['RSI'] < 70) &
    ~IndicatorFactory.adx_condition(df)
)
"""
BTC USDT 15m
Start                                               199
End                                              152446
Period                               1585 days 22:00:00
Start Value                                       100.0
End Value                                    339.885668
Total Return [%]                             239.885668
Benchmark Return [%]                         704.958516
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              34.136134
Max Drawdown Duration                 724 days 13:15:00
Total Trades                                       2068
Total Closed Trades                                2068
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  32.640232
Best Trade [%]                                11.809511
Worst Trade [%]                               -7.583474
Avg Winning Trade [%]                          1.023682
Avg Losing Trade [%]                          -0.399335
Avg Winning Trade Duration    0 days 03:12:49.333333333
Avg Losing Trade Duration     0 days 01:03:59.698492462
Profit Factor                                  1.242145
Expectancy                                     0.115999
Sharpe Ratio                                   1.353706
Calmar Ratio                                   0.952703
Omega Ratio                                     1.07672
Sortino Ratio                                  2.030376
dtype: object
Total Return:  2.398856677445716


Start                                               199
End                                               82412
Period                                856 days 09:30:00
Start Value                                       100.0
End Value                                    214.619382
Total Return [%]                             114.619382
Benchmark Return [%]                          92.375251
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              19.914819
Max Drawdown Duration                 312 days 13:45:00
Total Trades                                       1048
Total Closed Trades                                1048
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  31.965649
Best Trade [%]                                 8.501473
Worst Trade [%]                               -4.672585
Avg Winning Trade [%]                          0.957679
Avg Losing Trade [%]                          -0.335219
Avg Winning Trade Duration    0 days 03:22:12.537313432
Avg Losing Trade Duration     0 days 01:06:52.762973352
Profit Factor                                  1.363725
Expectancy                                      0.10937
Sharpe Ratio                                   1.798364
Calmar Ratio                                   1.931778
Omega Ratio                                    1.103622
Sortino Ratio                                  2.787165
dtype: object
Total Return:  1.1461938248696377
"""

'\nBTC USDT 15m\nStart                                               199\nEnd                                              152446\nPeriod                               1585 days 22:00:00\nStart Value                                       100.0\nEnd Value                                    339.885668\nTotal Return [%]                             239.885668\nBenchmark Return [%]                         704.958516\nMax Gross Exposure [%]                            100.0\nTotal Fees Paid                                     0.0\nMax Drawdown [%]                              34.136134\nMax Drawdown Duration                 724 days 13:15:00\nTotal Trades                                       2068\nTotal Closed Trades                                2068\nTotal Open Trades                                     0\nOpen Trade PnL                                      0.0\nWin Rate [%]                                  32.640232\nBest Trade [%]                                11.809511\nWorst Trade [%

In [ ]:
close_signal = (
    (df['SMA20'] < df['SMA50']) &

    (df['HMA'].shift(1) > df['HMA']) &
    ~IndicatorFactory.obv_condition(df) &

    ~IndicatorFactory.adx_condition(df)
)
#BTCUSDT
"""
        15 m
Start                                               199
End                                              152446
Period                               1585 days 22:00:00
Start Value                                       100.0
End Value                                    339.885668
Total Return [%]                             239.885668
Benchmark Return [%]                         704.958516
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              34.136134
Max Drawdown Duration                 724 days 13:15:00
Total Trades                                       2068
Total Closed Trades                                2068
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  32.640232
Best Trade [%]                                11.809511
Worst Trade [%]                               -7.583474
Avg Winning Trade [%]                          1.023682
Avg Losing Trade [%]                          -0.399335
Avg Winning Trade Duration    0 days 03:12:49.333333333
Avg Losing Trade Duration     0 days 01:03:59.698492462
Profit Factor                                  1.242145
Expectancy                                     0.115999
Sharpe Ratio                                   1.353706
Calmar Ratio                                   0.952703
Omega Ratio                                     1.07672
Sortino Ratio                                  2.030376
dtype: object
Total Return:  2.398856677445716

Start                                               199
End                                               82412
Period                                856 days 09:30:00
Start Value                                       100.0
End Value                                    214.619382
Total Return [%]                             114.619382
Benchmark Return [%]                          92.375251
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              19.914819
Max Drawdown Duration                 312 days 13:45:00
Total Trades                                       1048
Total Closed Trades                                1048
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  31.965649
Best Trade [%]                                 8.501473
Worst Trade [%]                               -4.672585
Avg Winning Trade [%]                          0.957679
Avg Losing Trade [%]                          -0.335219
Avg Winning Trade Duration    0 days 03:22:12.537313432
Avg Losing Trade Duration     0 days 01:06:52.762973352
Profit Factor                                  1.363725
Expectancy                                      0.10937
Sharpe Ratio                                   1.798364
Calmar Ratio                                   1.931778
Omega Ratio                                    1.103622
Sortino Ratio                                  2.787165
dtype: object
Total Return:  1.1461938248696377
"""

'\n        15 m\nStart                                               199\nEnd                                              152446\nPeriod                               1585 days 22:00:00\nStart Value                                       100.0\nEnd Value                                    339.885668\nTotal Return [%]                             239.885668\nBenchmark Return [%]                         704.958516\nMax Gross Exposure [%]                            100.0\nTotal Fees Paid                                     0.0\nMax Drawdown [%]                              34.136134\nMax Drawdown Duration                 724 days 13:15:00\nTotal Trades                                       2068\nTotal Closed Trades                                2068\nTotal Open Trades                                     0\nOpen Trade PnL                                      0.0\nWin Rate [%]                                  32.640232\nBest Trade [%]                                11.809511\nWorst Trade [%

In [ ]:
close_signal = (
    (df['SMA20'] < df['SMA50']) &

    (df['HMA'].shift(1) > df['HMA']) &
    ~IndicatorFactory.obv_condition(df) &

    ~IndicatorFactory.adx_condition(df)
)
# Sharpe Ratio                                   1.355024

              #BTCUSDT
"""
15m
Start                                               199
End                                              152446
Period                               1585 days 22:00:00
Start Value                                       100.0
End Value                                     503.60135
Total Return [%]                              403.60135
Benchmark Return [%]                         704.958516
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              49.172796
Max Drawdown Duration                1002 days 22:45:00
Total Trades                                        766
Total Closed Trades                                 766
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  36.553525
Best Trade [%]                                24.101391
Worst Trade [%]                               -6.653465
Avg Winning Trade [%]                          2.802182
Avg Losing Trade [%]                           -1.22004
Avg Winning Trade Duration    1 days 00:36:25.714285714
Avg Losing Trade Duration     0 days 11:25:37.037037037
Profit Factor                                  1.273935
Expectancy                                     0.526895
Sharpe Ratio                                   1.291384
Calmar Ratio                                    0.91662
Omega Ratio                                    1.039052
Sortino Ratio                                   1.85007
dtype: object
Total Return:  4.0360134979458815
"""

'\n15m \nStart                                               199\nEnd                                              152446\nPeriod                               1585 days 22:00:00\nStart Value                                       100.0\nEnd Value                                     503.60135\nTotal Return [%]                              403.60135\nBenchmark Return [%]                         704.958516\nMax Gross Exposure [%]                            100.0\nTotal Fees Paid                                     0.0\nMax Drawdown [%]                              49.172796\nMax Drawdown Duration                1002 days 22:45:00\nTotal Trades                                        766\nTotal Closed Trades                                 766\nTotal Open Trades                                     0\nOpen Trade PnL                                      0.0\nWin Rate [%]                                  36.553525\nBest Trade [%]                                24.101391\nWorst Trade [%]       

In [21]:




portfolio = vbt.Portfolio.from_signals(df['Close'], entries=long_signal, exits=close_signal ,sl_stop=0.002,  freq=timeperiod)
# pf.total_return()


# Get portfolio statistics
stats = portfolio.stats()
print(stats)
# Print the total return
total_return = portfolio.total_return()
print("Total Return: ", total_return)



Start                                       398
End                                       20719
Period                        846 days 18:00:00
Start Value                               100.0
End Value                            106.301113
Total Return [%]                       6.301113
Benchmark Return [%]                  45.030378
Max Gross Exposure [%]                    100.0
Total Fees Paid                             0.0
Max Drawdown [%]                      30.539132
Max Drawdown Duration         662 days 21:00:00
Total Trades                                104
Total Closed Trades                         104
Total Open Trades                             0
Open Trade PnL                              0.0
Win Rate [%]                          23.076923
Best Trade [%]                        17.106011
Worst Trade [%]                       -6.382979
Avg Winning Trade [%]                  3.679113
Avg Losing Trade [%]                  -0.974791
Avg Winning Trade Duration      0 days 0